# Imports

In [ ]:
## Logging and Shell
import logging
logging.basicConfig(
    level=logging.ERROR,
    force=True
)

## Numeric imports
import pandas as pd

## File I/O
from pathlib import Path
import json

# Cheminformatics
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

DIM    = 300
ASPECT = 3/2
IPythonConsole.molSize = (int(ASPECT*DIM), DIM)   # Change image size

# Custom imports
from polymerist.monomers import specification
from polymerist.rdutils.reactions import reactions

# Static Paths
RAW_DATA_DIR  = Path('raw_monomer_data')
PROC_DATA_DIR = Path('processed_monomer_data')
RXN_FILES_DIR = Path('poly_rxns')
# RXN_FILES_DIR = Path('rxn_smarts')

# Load monomer and rxn data 

In [ ]:
input_data_path = PROC_DATA_DIR / '20231114_polyid_data_density_FILTERED.csv'
df = pd.read_csv(input_data_path, index_col=0)

## Load pre-defined functional group SMARTS queries

In [ ]:
with Path('poly_rxns/fn_group_smarts.json').open('r') as file:
    fn_group_smarts = json.load(file)

fn_groups = {
    group_name : Chem.MolFromSmarts(smarts)
        for group_name, smarts in fn_group_smarts.items()
}

## Load pre-defined reactions with functional group and name backmap

In [ ]:
with (RXN_FILES_DIR / 'rxn_backmap.json').open('r') as file: # load table of functional group for each reaction
    rxn_backmap = json.load(file)

with (RXN_FILES_DIR / 'rxn_groups.json').open('r') as file: # load table of functional group for each reaction
    rxn_groups = json.load(file)

rxns = {
    rxnname : reactions.AnnotatedReaction.from_rxnfile(RXN_FILES_DIR / f'{rxnname}.rxn')
        for rxnname in rxn_backmap.values()
}

In [ ]:
whitelisted_rxns = ['amide', 'urethane', 'ester', 'carbonate']
df = df[df.mechanism.map(lambda s : s in whitelisted_rxns)]

## Group by mechanism

In [ ]:
keys = ['mechanism']

df_grouper = df.groupby(keys)
frames = {
    mech : df_grouper.get_group(mech)
        for mech in df_grouper.groups
}

# Polymerizing fragments

## Example for polymerization of individual molecule

In [ ]:
# mechname = 'ester'
mechname = 'amide'
# mechname = 'carbonate'
# mechname = 'vinyl'
# mechname = 'imide'

frame = frames[mechname]
row = frame.loc[frame.index[1]]

mechname = rxn_backmap[row['mechanism']]
print(mechname)

rxn_pathway = rxns[mechname]
named_reactants = {}
for i in (1, 2):
    reactant = Chem.MolFromSmiles(row[f'smiles_expanded_monomer_{i}'], sanitize=False)
    Chem.SanitizeMol(reactant, sanitizeOps=specification.SANITIZE_AS_KEKULE)
    display(reactant)
    named_reactants[ row[f'IUPAC_monomer_{i}'] ] = reactant

reactants = [i for i in named_reactants.values()]

In [ ]:
from polymerist.rdutils.amalgamation import portlib
from polymerist.rdutils.reactions import reactors

reactor = reactors.PolymerizationReactor(rxn_pathway)
for (dimer, pair) in reactor.propagate(reactants):
    display(dimer)
    for frag in pair:
        display(frag)

    print('='*50)

## Polymerize all SMILES into well-specified fragments with ports

In [ ]:
from polymerist.monomers import MonomerGroup
from polymerist.monomers import specification
from polymerist.genutils.containers import RecursiveDict


# MONO_INFO_DIR  : Path = Path('monomer_files')
MONO_INFO_DIR  : Path = Path('monomer_fragments')
MONO_INFO_DIR.mkdir(exist_ok=True)

frag_registry = RecursiveDict()
for (i, row) in df.iterrows():
    # look up reactive groups and pathway by chemistry
    mechname = rxn_backmap[row['mechanism']]
    rxn_pathway = rxns[mechname]

    chem_dir : Path = MONO_INFO_DIR / mechname
    chem_dir.mkdir(exist_ok=True)

    # load reactants with IUPAC names from chemical table
    named_reactants = {}
    for j in (1, 2):
        reactant = Chem.MolFromSmiles(row[f'smiles_expanded_monomer_{j}'], sanitize=False)
        Chem.SanitizeMol(reactant, sanitizeOps=specification.SANITIZE_AS_KEKULE)
        named_reactants[ row[f'IUPAC_monomer_{j}'] ] = reactant

    # polymer_name = f'poly({",".join(named_reactants.keys())})' # TODO : make sure this conforms to IUPAC standards for naming
    polymer_name = f'poly({"-co-".join(named_reactants.keys())})' # TODO : make sure this conforms to IUPAC standards for naming
    initial_reactants = [reactants for reactants in named_reactants.values()] # must convert to list
    frag_registry[mechname][i] = polymer_name

    # first round of polymerization (initiation)
    monogrp = MonomerGroup()
    reactor = reactors.PolymerizationReactor(rxn_pathway)
    for dimer, frags in reactor.propagate(initial_reactants):
        for assoc_group_name, rdfragment in zip(named_reactants.keys(), frags):
            # generate spec-compliant SMARTS
            raw_smiles = Chem.MolToSmiles(rdfragment)
            exp_smiles = specification.expanded_SMILES(raw_smiles)
            spec_smarts = specification.compliant_mol_SMARTS(exp_smiles)

            # record to monomer group
            affix = 'TERM' if MonomerGroup.is_terminal(rdfragment) else 'MID'
            monogrp.monomers[f'{assoc_group_name}_{affix}'] = [spec_smarts]

    monogrp.to_file(chem_dir / f'{polymer_name}.json')

with (MONO_INFO_DIR / 'dataset_backmap.json').open('w') as file:
    json.dump(frag_registry, file, indent=4)

# Building PDB files from monomer fragments

In [ ]:
from polymerist.maths.greek import GREEK_PREFIXES
from polymerist.polymers import building


master_pdb_dir = Path('pdb_files_new')
master_pdb_dir.mkdir(exist_ok=True)

DOP = 5

pdb_dir = master_pdb_dir / f'{GREEK_PREFIXES[DOP]}mers'
pdb_dir.mkdir(exist_ok=True)

failed = RecursiveDict()
for chemistry in rxn_groups.keys():
    mono_src_dir = MONO_INFO_DIR / chemistry
    if not mono_src_dir.exists():
        continue

    chem_dir = pdb_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mono_path in mono_src_dir.iterdir():
        monogrp = MonomerGroup.from_file(mono_path)
        try:
            polymer = building.build_linear_polymer(monomers=monogrp, DOP=DOP, sequence='AB')  
            savepath = chem_dir / f'{mono_path.stem}.pdb'
            building.mbmol_to_openmm_pdb(savepath, polymer)
            
            # polymer.save(asstrpath(savepath))
        except Exception as e:
            failed[DOP][chemistry][e.__class__.__name__][mono_path.stem] = monogrp

In [ ]:
failed